In [1]:

##
#
# -- Sparkmon pour Spark - Monster
#
# -- Script written by DIVAD Loïc: https://github.com/DivLoic
##
#%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
class sparkmon:
    """
        Objet  
    """
    
    def __init__(self, path, target):
        """initialising the object.
        """
        import pprint
        import json
        #import pyspark
        self.pp = pprint.PrettyPrinter(indent=4)
        
        self.rdd = sc.textFile("hdfs:/" + path)
        self.target = target
        
    def _feedBack(self, res, folder, prod):
        """ Utility function to display the result or store them in hdfs.
            Boolean Prod: False, display the result as list / true, store the result in hdfs as a text file.
            String folder: place to store the result in hdfs.
            spark RDD res: result of a spark .cach(), collect() or reduce()
        """
        if(prod):
            res.saveAsTextFile("hdfs:/" + self.target + folder)
        else:
            l = res.collect()
            self.pp.pprint(l)
        
    def crachCount(self, pivot=True, prod=False):
        """ Easest Map / Reduce ever 
            Boolean pivot: we consider (or not) the secound street of the fork
        """
        if(pivot):
            res = self.rdd.map(lambda line: line.split(";"))\
            .flatMap(lambda y: [(y[8],1),(y[9],1)])\
            .filter(lambda y: y[0] != '')\
            .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda y: y[1], ascending=False)
        else:
            res = self.rdd.map(lambda line: line.split(";"))\
            .map(lambda y: (y[8],1))\
            .filter(lambda y: y != '')\
            .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda y: y[1], ascending=False)
            
        self._feedBack(res, '/global/', prod)
        
        return res.collect()
        
        
    def byRingz(self, year="2012", prod=False):
        """ Analysis of the number of crash evert quarter.
            String year: fortamt "yyyy" for 2012 -> 12 / 0 for the entire scope.
        """
        pattern = 'BD PERIPHERIQUE'
        inter = 'INTERIEUR'
        exter = 'EXTERIEUR'
        
        res = self.rdd.map(lambda line: line.split(";"))\
            .filter(lambda y: y[0][:4] == year if year != '0' else y[0])\
            .filter(lambda y: y[8].find(pattern) != -1 or y[8].find(pattern) != -1)\
            .map(lambda y: (y[8]+'_'+y[3] ,1) if y[8].find(pattern)!= -1 else (y[9]+'_'+y[3],1))\
            .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda y: y[1])
            
        self._feedBack(res, '/ringz/', prod)    
        
    def byDept(self, year="2012", prod=False):
        """ Count the number of crach by dept.
            String year: the year support of the analysis. all the row availabale if scope = 0
        """
        res = self.rdd.map(lambda line: line.split(";"))\
            .filter(lambda y: y[0][:4] == year if year != '0' else y[0])\
            .filter(lambda y: y[0] != 'com')\
            .map(lambda y: (y[3], 1))\
            .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda y: y[1],ascending=False)
    
        self._feedBack(res, '/dept/', prod)
        
    def byCart(self, prod=False):
        """ We can also take the type of vehicule as axis
        """
        res = self.rdd.map(lambda line: line.split(";"))\
            .filter(lambda y: y[0] != '')\
            .map(lambda y: (y[10], 1))\
            .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda y: y[1],)
    
        self._feedBack(res, '/cart/', prod)
        
    def topThree(self, year='2012', prod=False):
        top = self.rdd.map(lambda line: line.split(";"))\
            .flatMap(lambda y: [(y[8],1),(y[9],1)])\
            .filter(lambda y: y[0].find('PERIPHERIQUE') == -1 and y[0] != '')\
            .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda y: y[1], ascending=False)\
            .collect()
            
        top = top[:3]   
        top = [t[0] for t in top]
        #print(top)
        
        res = self.rdd.map(lambda line: line.split(";"))\
                .filter(lambda y: y[0][:4] == year)\
                .flatMap(lambda y: [(y[8],y[0]),(y[9],y[0])])\
                .filter(lambda y: y[0] in top)\
                .map(lambda y: (y[0] + '_' + y[1][5:7], 1))\
                .reduceByKey(lambda a, b: a + b)\
                .sortBy(lambda y: y[0])
                
        self._feedBack(res, '/three/', prod)
                 
                 

In [ ]:
#.filter(lambda y: y[0][:3] == year)\
